# MURCHIE85 TWITTER STREAM PROCESSING 

![image](https://murchie85.github.io/images/index/fab.jpg)



This repo is for pulling metrics on a given keyword (best use keywords that are hashtagged) and performs a series of reporting and analysis.

## OVERVIEW

- Pulls live tweets from all over twitter 
- Imports Tweepy Library 
- Consumes


## Limitations & info
1. LOCATION filter is tricky
2. Streaming connects to the “public streams” (all public data) 

Example of parameter setting done in stream filter 

```
stream.filter(follow=["2211149702"])
```


# AUTHENTICATION

You will need to change f variable to point to your credentials file, I save mines in a text file and split the comma. Not uploaded to this repo obviously.

In [1]:
import datetime


f = open("../donotgit/access.txt", "r")
keys = f.read()
f.close()
keys  = keys.split(',')
#print("Access Keys are : " + str(keys))
ACCESS_TOKEN = keys[0]
ACCESS_SECRET = keys[1]


f = open("../donotgit/consumer.txt", "r")
keys = f.read()
keys  = keys.split(',')
#print("Consumer Keys are : " + str(keys))
CONSUMER_KEY = keys[0]
CONSUMER_SECRET = keys[1]


# PULLING DOWN THE DATA 

This is where the magic happens, api authentication is set up, tweepy library is imported for the work and a class allow us to pull live data. This is appended to array for processing later. 

If you want to change the parameters, edit the following line:  

```
stream.filter(track=["SexStrike"],languages=["en"])
```

*NOTE* -   this will just keep running until you force terminate, i haven't added in a terminate loop yet.

In [2]:
# Import the necessary package to process data in JSON format
try:
    import json
except ImportError:
    import simplejson as json

# Import the tweepy library
import tweepy

# Variables that contains the user credentials to access Twitter API 
# Captured earlier 

# Setup tweepy to authenticate with Twitter credentials:

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# Create the api to connect to twitter with your creadentials
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

status_array = []

class StreamListener(tweepy.StreamListener):

    def on_status(self, status):
        status_array.append(status._json)
        print(status.text)
        
    def on_error(self, status_code):
        if status_code == 420:
            return False

stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)

stream.filter(track=["SexStrike"],languages=["en"])

RT @RealCandaceO: From an evolutionary perspective, the radical feminist #sexstrike has the potential to save America. 

Yes, ladies—leave…
RT @usminority: This is the greatest tweet that has ever been put forth to humanity. 

My eyes will never again gaze upon anything on twitt…
RT @ericisaac: .@Alyssa_Milano calls for all women to withhold sex from men while she has no issue fucking the entire country by helping @J…
RT @LATiffani1: Go on a  #SexStrike Alyssa Milano. 

America can be unburdened from you and your liberal progressive hairy arm pitted pals…
RT @KristySwansonXO: Wow! Today’s my lucky day 😂🤣 https://t.co/oq1oOpGONW
RT @shannonrwatts: Rape has nothing to do with sex - it’s about hatred and power, the same causes of women being stripped of their rights.…
RT @usminority: Why do I get the feeling tens of millions of men are about to #Sexit the Democratic Party? https://t.co/XrKqv1iQMn
RT @PenceConscience: It is time for me too enter the battle. How many women over the years ha

RT @DadoftheDecade: There are an awful lot of triggered men on Twitter this morning complaining about women making their own choices about…
RT @RealCandaceO: From an evolutionary perspective, the radical feminist #sexstrike has the potential to save America. 

Yes, ladies—leave…
RT @AngelaBelcamino: My pussy, my choice. 
#SexStrike
RT @LoriinUtah: Since when do women not have legal control over their bodies @Alyssa_Milano? I will tell you when: When they are being brut…
RT @Evelynne24: I'm  in total agreement...no one should be having sex with @Alyssa_Milano or any of her type. We already have enough wackos…
RT @Alyssa_Milano: Our reproductive rights are being erased.

Until women have legal control over our own bodies we just cannot risk pregna…
RT @michellemalkin: What if Alyssa Milano called for a #SexStrike and no one came? 

Oh, wait. I guess that's her goal.

🤢 https://t.co/Axt…
RT @mostlybree: I'm going to pretend SexStrike isn't happening for 8,000 reasons large and small but p

RT @Alyssa_Milano: Our reproductive rights are being erased.

Until women have legal control over our own bodies we just cannot risk pregna…
RT @MADDABDOG: #sexstrike = repulsive liberal females and soy boy liberal males don’t reproduce...great idea from head dipshit @Alyssa_Mila…
RT @Tombx7M: The big winner in the sex strike is the unborn
 #SexStrike #Qanon #MAGA #KAG #Trump
#Cnnsotu #Mediabuzz #FoxAndFriends
#Mother…
Radical Feminists think people want to have sex with them!??!?!

🤣🤣🤣🤣🤣

It's like getting intimate with a piranha..… https://t.co/Fj5NxhDnDm
RT @mikeandersonsr: Some jokes write themselves. https://t.co/1zga3m9Vhv
RT @SarahTaber_bww: Yup. Folks really don't seem to understand how many conservative men have mistresses, sidechicks, or a rentboy habit. h…
RT @thecjpearson: I’m sorry but I’m not sure how consequential women who don’t shave their underarms and wear vaginas on their head partici…
RT @pinkk9lover: @whatznextfolks The only way they can do this 😂 @whatznextfolks 

KeyboardInterrupt: 

# PRINT AND SAVE DATA

This section allows me to pull the recently mined data into a file in raw format incase i need it later.

In [9]:
currentDT = datetime.datetime.now()
filename = "data/streamout-" + str(currentDT) + ".txt"
reportfile = "reports/printout-"+ str(currentDT) + ".txt"
print(filename)

with open(filename, 'w') as f:
    for item in status_array:
        f.write("%s\n" % item)
f.close()

data/streamout-2019-05-12 14:16:54.026646.txt


# STREAM METADATA

In [13]:
print('Number of records')
print(len(status_array))
print('')
print('The data keys are : ')
print(status_array[0].keys())
print('')
print('An example element looks like: ')

Number of records
150

The data keys are : 
dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

An example element looks like: 


# REPORT GENERATION

In [10]:
 
with open(reportfile, 'w') as f:
    for y in range(0, len(status_array)):
        f.write('************************************************************\n')
        f.write(status_array[y]['text'])
        f.write('------------------------------\n')
        f.write(status_array[y]['created_at'])
        f.write('------------------------------\n')
        f.write(status_array[y]['user']['name'])
        f.write('------------------------------\n')
        f.write(str(status_array[y]['user']['location']))
        f.write('------------------------------\n')
        f.write(str(status_array[y]['user']['description']))
        f.write('--------------REPLY COUNT----------------\n|')
        f.write(str(status_array[y]['reply_count']))
        f.write('--------------RETWEET COUNT----------------\n')
        f.write(str(status_array[y]['retweet_count']))
        f.write('--------------RETWEETED?----------------\n')
        f.write(str(status_array[y]['retweeted']))
        f.write('--------------RETWEET COUNT----------------\n')
        f.write(str(status_array[y]['retweet_count']))
        f.write('--------------FAVOURITED----------------\n')
        f.write(str(status_array[y]['favorited']))
        f.write('--------------FAVOURITE COUNT----------------\n')
        f.write(str(status_array[y]['favorite_count']))
        f.write('\n')
        f.write('\n')
        
f.close()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
for x in range(0, len(status_array)):
    print('************************************************************')
    print(status_array[x]['text'])
    print('------------------------------')
    print(status_array[x]['created_at'])
    print('------------------------------')
    print(status_array[x]['user']['name'])
    print('------------------------------')
    print(status_array[x]['user']['location'])
    print('------------------------------')
    print(status_array[x]['user']['description'])
    print('--------------REPLY COUNT----------------')
    print(status_array[x]['reply_count'])
    print('--------------RETWEET COUNT----------------')
    print(status_array[x]['retweet_count'])
    print('--------------RETWEETED?----------------')
    print(status_array[x]['retweeted'])
    print('--------------RETWEET COUNT----------------')
    print(status_array[x]['retweet_count'])
    print('--------------FAVOURITED----------------')
    print(status_array[x]['favorited'])
    print('--------------FAVOURITE COUNT----------------')
    print(status_array[x]['favorite_count'])
    print('')
    print('')


************************************************************
RT @RealCandaceO: From an evolutionary perspective, the radical feminist #sexstrike has the potential to save America. 

Yes, ladies—leave…
------------------------------
Sun May 12 13:07:14 +0000 2019
------------------------------
DebLF ☝☝
------------------------------
Pacific Northwest
------------------------------
Living among the Snowflakes on the Oregon Coast. Happily married to my best friend. NOT looking for dates.  #NRA  #KAG #Trump2020 #BUILDTHEWALL #2ThumbsUPTrain
--------------REPLY COUNT----------------
0
--------------RETWEET COUNT----------------
0
--------------RETWEETED?----------------
False
--------------RETWEET COUNT----------------
0
--------------FAVOURITED----------------
False
--------------FAVOURITE COUNT----------------
0


************************************************************
RT @usminority: This is the greatest tweet that has ever been put forth to humanity. 

My eyes will never again gaze

------------------------------
Sun May 12 13:07:41 +0000 2019
------------------------------
American Desert Flower 🇺🇸🌻
------------------------------
USA
------------------------------
God - Family - Country 🇺🇸
--------------REPLY COUNT----------------
0
--------------RETWEET COUNT----------------
0
--------------RETWEETED?----------------
False
--------------RETWEET COUNT----------------
0
--------------FAVOURITED----------------
False
--------------FAVOURITE COUNT----------------
0


************************************************************
RT @40_head: I support Alyssa Milano's #SexStrike

Think about it

#Abstinence, a #ConservativeVirtue, will teach liberals self control

Li…
------------------------------
Sun May 12 13:07:41 +0000 2019
------------------------------
Tom &amp; Teresa Hie
------------------------------
South GA
------------------------------
None
--------------REPLY COUNT----------------
0
--------------RETWEET COUNT----------------
0
--------------RETWEETED?--

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun May 12 13:07:14 +0000 2019',
 'entities': {'hashtags': [{'indices': [73, 83], 'text': 'sexstrike'}],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 878247600096509952,
    'id_str': '878247600096509952',
    'indices': [3, 16],
    'name': 'Candace Owens',
    'screen_name': 'RealCandaceO'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1127560866935656450,
 'id_str': '1127560866935656450',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'retweeted_status': {'contributors': None,
  'coordinates': None,
  'created_at': 'Sat May 11 19:05:05 +0000 2019',
  'entities': {'hashtags': [{'indices': [55, 65], 'text': 'sexstr